In [ ]:
pip install transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%pip install accelerate peft bitsandbytes transformers trl

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset,Dataset
import transformers

In [ ]:
from random import randrange
dataset = load_dataset("Yijia-Xiao/pii-wikidoc_patient_information",split="train")

print(f"dataset size: {len(dataset)}")

# 'randrange' is a function from the 'random' module that generates a random number within the specified range.
print(dataset[randrange(len(dataset))])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5942 [00:00<?, ? examples/s]

dataset size: 5942
{'output': 'Most people recover from symptoms of the infection, even without treatment, although they may continue to carry the worms in their body. Complications may be caused by adult worms that move to certain organs such as the bile duct, pancreas, or appendix, or multiply and cause a blockage in the intestine.', 'instruction': 'Answer this question truthfully', 'input': 'What to expect if I have Ascariasis  (Outlook/Prognosis)?', 'cleaned_output': 'Most people recover from symptoms of the infection, even without treatment, although they may continue to carry the worms in their body. {{NAME}} may be caused by adult worms that move to certain organs such as the bile duct, pancreas, or appendix, or multiply and cause a blockage in the intestine.'}


In [ ]:
dataset = dataset.remove_columns(['cleaned_output'])
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 5942
})

In [ ]:
df = pd.DataFrame(dataset)

df = df[['instruction', 'input', 'output']]

rearranged_dataset = Dataset.from_pandas(df)
rearranged_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5942
})

In [ ]:
model_name="EleutherAI/gpt-neo-125m"

In [ ]:
tokenizer_id = model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, use_fast=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

GPT2TokenizerFast(name_or_path='EleutherAI/gpt-neo-125m', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
# 'create_message_column' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'messages' and the value is a list of dictionaries, each representing a message.

# 'row' is the input to the 'create_message_column' function. It is expected to be a dictionary with keys 'instruction', 'input', and 'output'.

# 'messages' is a list that will contain the messages.

# 'user' is a dictionary that represents a user message. The 'content' key contains the instruction and input from the row, and the 'role' key is set to 'user'.

# 'messages.append(user)' adds the user message to the 'messages' list.

# 'assistant' is a dictionary that represents an assistant message. The 'content' key contains the output from the row, and the 'role' key is set to 'assistant'.

# 'messages.append(assistant)' adds the assistant message to the 'messages' list.

# 'return {"messages": messages}' returns a dictionary with a single key-value pair. The key is 'messages' and the value is the 'messages' list.

# 'format_dataset_chatml' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row.

# 'row' is the input to the 'format_dataset_chatml' function. It is expected to be a dictionary with a key 'messages'.

# 'return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}' returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row. The 'add_generation_prompt' parameter is set to False, which means that no generation prompt will be added to the end of the text. The 'tokenize' parameter is set to False, which means that the text will not be tokenized.
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruction']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

# def format_dataset_chatml(row):
#     return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


def format_dataset_chatml(row):
    text = tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)
    input_ids = tokenizer(text, padding='max_length', truncation=True, max_length=1024, return_tensors='pt').input_ids  # Adjust max_length
    return {"input_ids": input_ids}

In [ ]:
dataset_chatml = rearranged_dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/5942 [00:00<?, ? examples/s]

Map:   0%|          | 0/5942 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


In [ ]:
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'messages', 'input_ids'],
        num_rows: 5644
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'messages', 'input_ids'],
        num_rows: 298
    })
})

In [ ]:
from peft import LoraConfig
from trl import SFTTrainer
import os
import torch
from transformers import (
    BitsAndBytesConfig,
    logging,
)

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,

)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
print(model)

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (v_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (q_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (out_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"],
)

In [ ]:
training_params = transformers.TrainingArguments(
    "gpt-neoMedChatbot",
    num_train_epochs=3,
    do_eval=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=100,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    peft_config=peft_params,
    dataset_text_field="input_ids",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,3.187400,3.011765
200,2.875600,2.822810
300,2.713400,2.735769
400,2.694800,2.683273
500,2.638600,2.644074
600,2.652500,2.614975
700,2.624200,2.585567
800,2.644400,2.570129
900,2.600700,2.554037
1000,2.462000,2.541805


Step,Training Loss,Validation Loss
100,3.187400,3.011765
200,2.875600,2.822810
300,2.713400,2.735769
400,2.694800,2.683273
500,2.638600,2.644074
600,2.652500,2.614975
700,2.624200,2.585567
800,2.644400,2.570129
900,2.600700,2.554037
1000,2.462000,2.541805


TrainOutput(global_step=4233, training_loss=2.5398848939386913, metrics={'train_runtime': 5263.4986, 'train_samples_per_second': 3.217, 'train_steps_per_second': 0.804, 'total_flos': 9152314711474176.0, 'train_loss': 2.5398848939386913, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neoMedChatbot/commit/68861b7f097f3003068dcb52ac776034da9d0973', commit_message='End of training', commit_description='', oid='68861b7f097f3003068dcb52ac776034da9d0973', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.model.push_to_hub("gpt-neoMedChatbot")

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neoMedChatbot/commit/154293064f3a1db3a36198c0bd9050c71fd80818', commit_message='Upload model', commit_description='', oid='154293064f3a1db3a36198c0bd9050c71fd80818', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.model.push_to_hub("gpt-neo-MedChatbot")

adapter_model.safetensors:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neo-MedChatbot/commit/5590759025d62922e5264babeb03ab44038bd8e7', commit_message='Upload model', commit_description='', oid='5590759025d62922e5264babeb03ab44038bd8e7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.model.save_pretrained("/content/drive/MyDrive/finetuning/gpt-neo medical chatbot")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
output_dir="/content/drive/MyDrive/finetuning/gpt-neo-",

In [ ]:
trainer.save_model("/content/drive/MyDrive/finetuning/gpt-neo-")
trainer.save_state()
tokenizer.save_pretrained("/content/drive/MyDrive/finetuning/gpt-neo-")

('/content/drive/MyDrive/finetuning/gpt-neo-/tokenizer_config.json',
 '/content/drive/MyDrive/finetuning/gpt-neo-/special_tokens_map.json',
 '/content/drive/MyDrive/finetuning/gpt-neo-/vocab.json',
 '/content/drive/MyDrive/finetuning/gpt-neo-/merges.txt',
 '/content/drive/MyDrive/finetuning/gpt-neo-/added_tokens.json',
 '/content/drive/MyDrive/finetuning/gpt-neo-/tokenizer.json')

In [ ]:
tokenizer.push_to_hub("gpt-neo-MedChatbot")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neo-MedChatbot/commit/edc5a48c87d7b0aa0f3321d4967c46a0c5731c9a', commit_message='Upload tokenizer', commit_description='', oid='edc5a48c87d7b0aa0f3321d4967c46a0c5731c9a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import json
with open('/content/drive/MyDrive/finetuning/gpt-neo-/training_args.json', 'w') as f:
    json.dump(training_params.to_dict(), f)

In [ ]:
from huggingface_hub import Repository

In [ ]:
repo = Repository(local_dir='/content/drive/MyDrive/finetuning/gpt-neo-', clone_from="gpt-neo-MedChatbot")
repo.push_to_hub()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('/content/drive/MyDrive/finetuning/gpt-neo-'). If you really want to do this, do it manually:
 cd /content/drive/MyDrive/finetuning/gpt-neo- && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [ ]:
trainer.push_to_hub("gpt-neo-MedChatbot")

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neoMedChatbot/commit/4fb5117971b5a3e7ed0076aca9c26c8f007b4161', commit_message='gpt-neo-MedChatbot', commit_description='', oid='4fb5117971b5a3e7ed0076aca9c26c8f007b4161', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.model.push_to_hub("gpt-neoMedChatbot")

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neoMedChatbot/commit/040e1ada539d2f64086ad13bbbbb1718b06fd4f1', commit_message='Upload model', commit_description='', oid='040e1ada539d2f64086ad13bbbbb1718b06fd4f1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

model_id = 'ajsal27/gpt-neo-MedChatbot'

text_generator = pipeline('text-generation', model=model_id)

prompt = "The future of medicine involves"
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
generated_text


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The future of medicine involves a number of issues. The most important of these is the need to understand the role of the immune system in the development of disease. The immune system is a complex system that is composed of a number of cells that are responsible for the immune response. The immune system is a complex system that is composed of a number of cells that are responsible for the immune response. The immune system is a complex system that is composed of a number of cells that are responsible for the immune response'}]

In [ ]:
model_id = 'ajsal27/gpt-neo-MedChatbot'

text_generator = pipeline('text-generation', model=model_id)

prompt = "symptoms of allergy"
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
generated_text

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "symptoms of allergy to the skin, and the skin's own immune system.\n\nThe skin's own immune system is the body's defense against the harmful effects of the sun.\n\nThe skin's own immune system is the body's defense against the harmful effects of the sun.\n\nThe skin's own immune system is the body's defense against the harmful effects of the sun.\n\nThe skin's own immune system is the body's defense against the harmful effects of the sun."}]

In [ ]:
prompt = "symptoms of cancer"
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
generated_text

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'symptoms of cancer, and the treatment of cancer.\n\nThe treatment of cancer is a complex process that involves many different types of cancer. The most common type of cancer is cancer of the breast, colon, and lung. The most common type of cancer is cancer of the prostate, colon, and lung. The treatment of cancer is a complex process that involves many different types of cancer. The most common type of cancer is cancer of the prostate, colon, and lung. The treatment of cancer'}]

In [ ]:
model.push_to_hub("gpt-neo-MedChatbot")

model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neo-MedChatbot/commit/4c6478424bc4197f373b79898b2e76e7ce5b57ef', commit_message='Upload GPTNeoForCausalLM', commit_description='', oid='4c6478424bc4197f373b79898b2e76e7ce5b57ef', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.evaluate()

{'eval_loss': 2.407747268676758,
 'eval_runtime': 32.4205,
 'eval_samples_per_second': 9.192,
 'eval_steps_per_second': 2.313,
 'epoch': 3.0}

In [ ]:
model.push_to_hub("gpt-neoMedChatbot")

model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/gpt-neoMedChatbot/commit/fbf379eba17cd2a0730c4eaadf0de8891b98cf87', commit_message='Upload GPTNeoForCausalLM', commit_description='', oid='fbf379eba17cd2a0730c4eaadf0de8891b98cf87', pr_url=None, pr_revision=None, pr_num=None)